In [17]:
import numpy as np
import networkx as nx
import cvxpy as cp
import pickle

# Reading the Saved Random Graph
with open('random_graph.pkl', 'rb') as f:
    G, n, p = pickle.load(f)

# Retrieving Node Weights
w = {str(node): G.nodes[node]['weight'] for node in G.nodes()}

print(f'The Vertex Set of the Graph Is：{list(G.nodes)}')
print(f'The Edge Set of the Graph Is：{list(G.edges)}')
print(f'Node Weights: {w}')

# Greedy Algorithm
def greedy_mis(graph, w):
    i_set = set()
    sorted_graph = sorted(graph, key=lambda node: w[str(node)], reverse=True)
    for v in sorted_graph:
        i_set.add(v)
        if len(graph[v]) != 0:
            for w in graph[v]:
                if w in sorted_graph:
                    sorted_graph.remove(w)
    return i_set

# Converting Graph to Adjacency Matrix
nodes = list(G.nodes())
num_nodes = len(nodes)
adj = np.zeros((num_nodes, num_nodes), dtype=int)
for edge in G.edges():
    node1, node2 = edge
    index1 = nodes.index(node1)
    index2 = nodes.index(node2)
    adj[index1, index2] = 1
    adj[index2, index1] = 1

# Converting Adjacency Matrix to Adjacency List
graph = dict([])
for i in range(num_nodes):
    graph[i] = []
    for j in range(num_nodes):
        if adj[i, j] == 1:
            graph[i].append(j)

# Solving Maximum Independent Set Using Greedy Algorithm
greedy_is = greedy_mis(graph, w)
n = len(G.nodes)
binary_representation = [1 if i in greedy_is else 0 for i in range(n)]
binary_representation.reverse()
binary_string = ''.join(map(str, binary_representation))
A1 = binary_string
a1 = len(greedy_is)
Lower = 0
for i in range(len(greedy_is)):
    Lower += w[str(greedy_is.pop())]
print('Lower Bound Found by the Greedy Algorithm：', A1)
print('The Number of Lower Bound Points K1 Found by the Greedy Algorithm Is：', a1)
print('The Lower Bound Weight Found by the Greedy Algorithm Is：', Lower)

# Obtaining the Upper Bound a2 Using the Lovász Function
C = np.ones((len(G.nodes), len(G.nodes))) * max(w.values())
X = cp.Variable((n, n), PSD=True)
constraints = [cp.trace(X) == 1]
for i in range(len(adj)):
    for j in range(len(adj[0])):
        if adj[i][j] == 1:
            constraints.append(X[i][j] == 0)
prob = cp.Problem(cp.Maximize(cp.trace(C @ X)), constraints)
prob.solve()
print(X.value)
Upper = round(prob.value)
a2 = Upper // max(w.values())
print('The Number of Upper Bound Points Kl Found by Semidefinite Programming Is：', a2)
print('The Upper Bound Weight Found by Semidefinite Programming Is：', Upper)

The Vertex Set of the Graph Is：[0, 1, 2, 3, 4, 5, 6, 7]
The Edge Set of the Graph Is：[(0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (1, 3), (1, 4), (1, 6), (1, 7), (2, 3), (2, 7), (3, 4), (3, 6), (4, 6), (4, 7), (5, 6), (5, 7), (6, 7)]
Node Weights: {'0': 11, '1': 1, '2': 5, '3': 19, '4': 17, '5': 17, '6': 1, '7': 17}
Lower Bound Found by the Greedy Algorithm： 00101000
The Number of Lower Bound Points K1 Found by the Greedy Algorithm Is： 2
The Lower Bound Weight Found by the Greedy Algorithm Is： 36
[[-2.81136101e-07 -4.53553266e-08 -2.95146012e-11  2.08289075e-13
  -3.64697901e-12 -2.95146018e-11  2.08288937e-13  2.08295193e-13]
 [-4.53553266e-08  1.64521389e-01  1.64522159e-01  1.12838351e-11
   1.79827667e-10  1.64522162e-01  1.20705464e-11 -4.97942049e-12]
 [-2.95146012e-11  1.64522159e-01  3.33334203e-01  2.33175750e-11
   1.68812074e-01  3.33334494e-01  5.34325252e-08 -1.10589629e-11]
 [ 2.08289075e-13  1.12838351e-11  2.33175750e-11 -2.81136110e-07
   1.22420157e-11  4.91841212